#### 수질 음용성 여부 데이터
데이터 설명 : 수질 음용성 여부 (Potablillity 컬럼 : 0 ,1 )

---

train, val 데이터로 score를 계산했을 때는 0.68 정도로 비교적 낮은 점수가 나왔으나, <br>
평가용 데이터로 score를 계산한 결과 0.94 정도로 높은 점수가 나왔다. 

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,0,8.662710,173.531947,20333.079495,5.636388,439.787938,459.633120,16.283311,89.924253,5.120103
1,1,NaN,226.270824,15380.124079,6.661474,NaN,392.558205,14.083110,50.286395,4.516870
2,2,7.583770,217.283262,36343.407055,8.532726,375.964391,393.877683,17.442301,77.722257,3.642289
3,3,6.584813,182.375456,24723.106296,6.238920,NaN,414.350751,17.582615,78.213738,4.404132
4,4,7.179864,180.854211,10859.553752,8.263503,341.302486,358.056264,12.065317,83.329918,3.878447


,ID,Potability
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


In [2]:
# 결측치 확인
x_train.isnull().sum()

ID                   0
ph                 395
Hardness             0
Solids               0
Chloramines          0
Sulfate            617
Conductivity         0
Organic_carbon       0
Trihalomethanes    132
Turbidity            0
dtype: int64

In [3]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2620 entries, 0 to 2619
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2620 non-null   int64  
 1   ph               2225 non-null   float64
 2   Hardness         2620 non-null   float64
 3   Solids           2620 non-null   float64
 4   Chloramines      2620 non-null   float64
 5   Sulfate          2003 non-null   float64
 6   Conductivity     2620 non-null   float64
 7   Organic_carbon   2620 non-null   float64
 8   Trihalomethanes  2488 non-null   float64
 9   Turbidity        2620 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 204.8 KB


In [4]:
# ph, Sulfate, Trihalomethanes의 결측치를 각각의 평균으로 채움.

ph_mean = x_train.ph.mean()
Sulfate_mean = x_train.Sulfate.mean()
Trihalomethanes_mean = x_train.Trihalomethanes.mean()

x_train.ph.fillna(ph_mean, inplace=True)
x_test.ph.fillna(ph_mean, inplace=True)
x_train.Sulfate.fillna(Sulfate_mean, inplace=True)
x_test.Sulfate.fillna(Sulfate_mean, inplace=True)
x_train.Trihalomethanes.fillna(Trihalomethanes_mean, inplace=True)
x_test.Trihalomethanes.fillna(Trihalomethanes_mean, inplace=True)

In [5]:
# ID 컬럼 제거
x_train_drop = x_train.drop(columns=['ID'])
x_test_drop = x_test.drop(columns=['ID'])
y = y_train.Potability

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

rf = RandomForestClassifier()
xgb = XGBClassifier()

X_train, X_val, Y_train, Y_val = train_test_split(x_train_drop, y, test_size=0.2, random_state=47)

rf.fit(X_train, Y_train)
xgb.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [7]:
# 랜덤포레스트 모델 평가
print(accuracy_score(Y_train, rf.predict(X_train)))
# 검증
print(accuracy_score(Y_val, rf.predict(X_val)))
# XGboost 모델 평가
print(accuracy_score(Y_train, xgb.predict(X_train)))
# 검증
print(accuracy_score(Y_val, xgb.predict(X_val)))

1.0
0.6851145038167938
1.0
0.6221374045801527


In [8]:
pd.DataFrame({'ID' : x_test.ID, 'Potability' : rf.predict(x_test_drop)}).to_csv('000000000.csv', index=False)

In [9]:
result = pd.read_csv('000000000.csv')
result

,ID,Potability
0,16,0
1,20,1
2,23,1
3,37,0
4,38,0
...,...,...
651,3267,0
652,3269,0
653,3272,1
654,3273,1


In [10]:
# 평가용 데이터
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/y_test.csv")
y_test

,ID,Potability
0,16,0
1,20,1
2,23,1
3,37,0
4,38,0
...,...,...
651,3267,0
652,3269,0
653,3272,1
654,3273,1


In [11]:
rf.score(x_test_drop, y_test['Potability'])

0.9435975609756098